In [1]:
# title: MapBiomas Soil
# subtitle: Prepare environmental covariates
# author: Alessandro Samuel-Rosa and Taciara Zborowski Horst
# data: 2025

# Import necessary libraries
import os
import pandas as pd

# Identify working directory, saving the path to a variable
src_dir = os.getcwd()
work_dir = os.path.dirname(src_dir)

# Read the TXT file 'data/12_soildata.txt' with the soil data from the 'data' folder processed
# in the previous script '12_prepare_soil_covars.R'
# Field separator: tab
file_path = os.path.join(work_dir, 'data', '12_soildata.txt')
soildata_df = pd.read_csv(file_path, sep='\t', low_memory=False)
print(soildata_df.shape)
# (54555, 66)

(54555, 66)


In [ ]:
# Read the TXT file 'data/13_soildata.txt' with the coordinates from the 'data' folder
# Field separator: tab. Decimal separator: comma
# file_path = os.path.join(work_dir, 'data', '13_soildata.txt')
# df2 = pd.read_csv(file_path, sep='\t', low_memory=False)
# print(df2.shape)

# Compare the two data frames to check if there were changes in the data
# Use equals() method to compare the two data frames
# The result is a boolean value, True if the data frames are equal, False otherwise
# We consider only the following variables in the comparison:
# dataset_id, observacao_id, coord_x, coord_y, data_ano
# df1 = soildata_df[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
# df2 = df2[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
# if df1.equals(df2):
#     print("The DataFrames are identical.")
# else:
#     print("The DataFrames are different.")
# del df1, df2
# 29683 layers (GET FROM PREVIOUS SCRIPT)

In [2]:
# UNIQUE EVENTS WITH GEOGRAPHIC COORDINATES

# Filter out soil layers without geographic coordinates
# coord_x and coord_y are the columns with the geographic coordinates
soildata_xy = soildata_df[soildata_df['coord_x'].notnull()]
soildata_xy = soildata_xy[soildata_xy['coord_y'].notnull()]

# Remove all duplicates based on the following columns:
# "dataset_id", "observacao_id", "coord_x", "coord_y", "data_ano"
# The first occurrence is kept, and the others are removed
soildata_xy = soildata_xy[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
soildata_xy = soildata_xy.drop_duplicates()
target = 16569 # GET VALUE FROM PREVIOUS SCRIPT!
print(
  'There should be', target, 'events:', target == soildata_xy.shape[0],
  '\nThere are', soildata_xy.shape[0], 'events')

There should be 16569 events: True 
There are 16569 events


In [3]:
# Print the first and last 5 rows of the data frame
print(soildata_xy)

      dataset_id observacao_id    coord_x    coord_y  data_ano
0        ctb0001    CC-A1-2012 -52.468449 -31.812608    2012.0
2        ctb0001    CC-A1-2013 -52.468449 -31.812608    2013.0
4        ctb0001    CC-A2-2012 -52.468608 -31.812633    2012.0
6        ctb0001    CC-A2-2013 -52.468608 -31.812633    2013.0
8        ctb0001    CC-A3-2012 -52.468300 -31.812582    2012.0
...          ...           ...        ...        ...       ...
54532    ctb0838            74 -43.997580 -22.564130    2014.0
54536    ctb0838            75 -43.994350 -22.522300    2014.0
54540    ctb0838            78 -43.989760 -22.530160    2014.0
54544    ctb0838            88 -44.007310 -22.531180    2014.0
54550    ctb0838            89 -44.006970 -22.530190    2014.0

[16569 rows x 5 columns]


In [4]:
# Import necessary libraries
import ee
import geemap
import numpy as np  # Import numpy to access np.nan

# Initialize the Earth Engine API
# ee.Authenticate()
ee.Initialize(project='mapbiomas-solos-workspace')

# 1. Clean the DataFrame: Replace all NaN values with None
soildata_xy_cleaned = soildata_xy.replace({np.nan: None})

# 2. Convert the *cleaned* DataFrame to an Earth Engine Feature Collection
soildata_fc = geemap.df_to_ee(soildata_xy_cleaned, latitude='coord_y', longitude='coord_x')

# Function to split a feature collection (sampling points) into chunks
def split_sampling_points(fc, chunk_size):
    features = fc.toList(fc.size())
    chunks = [features.slice(i, i + chunk_size) for i in range(0, features.size().getInfo(), chunk_size)]
    return [ee.FeatureCollection(chunk) for chunk in chunks]

# Split the sample points into subsets of 1000 points each
# This is necessary to avoid timeout errors
chunk_size = 1000
soildata_chunks = split_sampling_points(soildata_fc, chunk_size)
print(len(soildata_chunks), 'chunks')

/home/alessandrorosa/projects/MapBiomas/mapbiomas-soil-train-prep/.virtualenvs/GEE/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


17 chunks


In [14]:
# SoilGrids 250m (Hengl et al., 2017)
# WRB Soil Reference Groups at 250-m spatial resolution
wrb_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/WRB_ALL_SOILS_SOILGRIDS_30M_GAPFILL"
wrb_bands = ['Ferralsols', 'Histosols', 'Nitisols', 'Vertisols', 'Plinthosols', 'Arenosols',
             'Podzols', 'Chernozems', 'Phaeozems', 'Umbrisols', 'Leptosols', 'Regosols',
             'Gleysols', 'Planosols', 'Stagnosols', 'Alisols', 'Luvisols', 'Acrisols', 'Lixisols']
wrb_image = ee.Image(wrb_path).select(wrb_bands)
print(wrb_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'Ferralsols', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [201550, 161033], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -76.316824120016, 0, -0.0002694945852358564, 7.772762827372571]}, {'id': 'Histosols', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [201550, 161033], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -76.316824120016, 0, -0.0002694945852358564, 7.772762827372571]}, {'id': 'Nitisols', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [201550, 161033], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -76.316824120016, 0, -0.0002694945852358564, 7.772762827372571]}, {'id': 'Vertisols', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [201550, 161033], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945

In [16]:
# Black Soils (FAO, 2022)
# Probability of occurrence of black soils at 1-km spatial resolution
black_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/FAO_2022_BLACKSOIL_1KM"
black_image = ee.Image(black_path)
print(black_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -73.98327050645872, 0, -0.0002694945852358564, 5.269697119701936]}], 'version': 1742421350068911, 'id': 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/FAO_2022_BLACKSOIL_1KM', 'properties': {'modeled_with': 'SOILGRIDS-ISRIC', 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-58.82043133238101, -33.75136753101224], [-53.70742644104491, -33.75136756456261], [-46.12607423143504, -33.75136756282918], [-40.83675875796949, -33.75136757159017], [-34.13695910007781, -33.75136755099094], [-28.847318949928372, -33.75136735808534], [-28.847372557145455, 5.269832232595247], [-34.489580175078395, 5.269832329090754], [-43.305105972802636, 5.2698323446237385], [-51.415389678547186, 5.269832321941617], [-57.40994723560988, 5.269832362732201], [-67.2833361050988, 5.2698323

In [17]:
# SoilGrids 250m (Poggio et al., 2021)
# Soil properties at 250-m spatial resolution at six standard depths (0-5cm, 5-15cm, 15-30cm,
# 30-60cm, 60-100cm, 100-200cm)

# Soil bulk density (bdod_mean)
bdod_path = "projects/soilgrids-isric/bdod_mean"
bdod_bands = ['bdod_0-5cm_mean', 'bdod_5-15cm_mean', 'bdod_15-30cm_mean',
              'bdod_30-60cm_mean', 'bdod_60-100cm_mean', 'bdod_100-200cm_mean']
bdod_image = ee.Image(bdod_path).select(bdod_bands)

# Clay content (clay_mean)
clay_path = "projects/soilgrids-isric/clay_mean"
clay_bands = ['clay_0-5cm_mean', 'clay_5-15cm_mean', 'clay_15-30cm_mean',
              'clay_30-60cm_mean', 'clay_60-100cm_mean', 'clay_100-200cm_mean']
clay_image = ee.Image(clay_path).select(clay_bands)

# Sand content (sand_mean)
sand_path = "projects/soilgrids-isric/sand_mean"
sand_bands = ['sand_0-5cm_mean', 'sand_5-15cm_mean', 'sand_15-30cm_mean', 
              'sand_30-60cm_mean', 'sand_60-100cm_mean', 'sand_100-200cm_mean']
sand_image = ee.Image(sand_path).select(sand_bands)

# Soil organic carbon (soc_mean)
soc_path = "projects/soilgrids-isric/soc_mean"
soc_bands = ['soc_0-5cm_mean', 'soc_5-15cm_mean', 'soc_15-30cm_mean',
             'soc_30-60cm_mean', 'soc_60-100cm_mean', 'soc_100-200cm_mean']
soc_image = ee.Image(soc_path).select(soc_bands)

# Volume of coarse fragments (cfvo_mean)
cfvo_path = "projects/soilgrids-isric/cfvo_mean"
cfvo_bands = ['cfvo_0-5cm_mean', 'cfvo_5-15cm_mean', 'cfvo_15-30cm_mean',
              'cfvo_30-60cm_mean', 'cfvo_60-100cm_mean', 'cfvo_100-200cm_mean']
cfvo_image = ee.Image(cfvo_path).select(cfvo_bands)

# Stack all soil property images into a single image
soil_image = (bdod_image
              .addBands(clay_image)
              .addBands(sand_image)
              .addBands(soc_image)
              .addBands(cfvo_image))
print(soil_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'bdod_0-5cm_mean', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [143495, 60728], 'crs': 'PROJCS["World_Mollweide", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["D_WGS_1984", \n      SPHEROID["WGS_1984", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]', 'crs_transform': [250, 0, -17960089.445510544, 0, -250, 8697788.131638981]}, {'id': 'bdod_5-15cm_mean', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [143495, 60728], 'crs': 'PROJCS["World_Mollweide", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["D_WG

In [20]:
# Geomorphometry

# Digital Elevation Model (DEM)
dem_path = "MERIT/DEM/v1_0_3"
dem_image = ee.Image(dem_path).select('dem').round()

# Land surface variables (Amatulli et al., 2020, 2018)
land_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m"

# Altitude
altitude_image = ee.Image(land_path).select('elevation').round()

# Slope
slope_image = ee.Image(land_path).select('slope').multiply(3.141593).divide(180).tan().multiply(100).round()

# Convergence
convergence_image = ee.Image(land_path).select('convergence').round()

# Compound Topographic Index (CTI)
# cti * 10
cti_image = ee.Image(land_path).select('cti').multiply(10).round()

# Eastness
# eastness * 100
eastness_image = ee.Image(land_path).select('eastness').multiply(100).round()

# Northness
# northness * 100
northness_image = ee.Image(land_path).select('northness').multiply(100).round()

# Profile Curvature
# pcurv * 1000
pcurv_image = ee.Image(land_path).select('pcurv').multiply(1000).round()

# Roughness
roughness_image = ee.Image(land_path).select('roughness').round()

# Stream Power Index (SPI)
spi_image = ee.Image(land_path).select('spi').add(1).log10().multiply(100).round()

# Elevation Standard Deviation
elev_stdev_image = ee.Image(land_path).select('elev_stdev').round()

# Maximum Multiscale Deviation
dev_magnitude_image = ee.Image(land_path).select('dev_magnitude').round()

# Geomorphon (geomorphological forms)
# geom
geom_image = ee.Image(land_path).select('geom').round()

# Stack all land surface variable images into a single image
dem_image = (dem_image
             .addBands(slope_image)
             .addBands(convergence_image)
             .addBands(cti_image)
             .addBands(eastness_image)
             .addBands(northness_image)
             .addBands(pcurv_image)
             .addBands(roughness_image)
             .addBands(spi_image)
             .addBands(elev_stdev_image)
             .addBands(dev_magnitude_image)
             .addBands(geom_image))
print(dem_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'dem', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [432000, 174000], 'crs': 'EPSG:4326', 'crs_transform': [0.0008333333333333334, 0, -180.00041666666667, 0, -0.0008333333333333334, 84.99958333333333]}, {'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [74216, 111325], 'crs': 'EPSG:4326', 'crs_transform': [0.0008084837557075694, 0, -90.00122016912233, 0, -0.0008084837557075694, 30.002023690552193]}, {'id': 'convergence', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [74216, 111325], 'crs': 'EPSG:4326', 'crs_transform': [0.0008084837557075694, 0, -90.00122016912233, 0, -0.0008084837557075694, 30.002023690552193]}, {'id': 'cti', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [74216, 111325], 'crs': 'EPSG:4326', 'crs_transform': [0.0008084837557075694, 0, -90.00122016912233, 0, -0.0008084837557075694, 30.002023690552193]}, {'id': 'eastn

In [23]:
# Structural Provinces (IBGE, 2019c)

# First level: Provinces
province_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/IBGE_PROVINCIAIS_ESTRUTURAIS_250mil_2025/IBGE_PROVINCIAS_250MIL_DUMMY"
province_image = ee.Image(province_path)

# Second level: Subprovinces
subprovince_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/IBGE_PROVINCIAIS_ESTRUTURAIS_250mil_2025/subprovincias"
subprovince_image = ee.Image(subprovince_path)

# Stack all structural province images into a single image
geo_image = (province_image
             .addBands(subprovince_image))
print(geo_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'Amazonas_Solimoes_Provincia', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -73.98327050645874, 0, -0.00026949458523585647, 5.269697119701937]}, {'id': 'Amazonia_Provincia', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -73.98327050645874, 0, -0.00026949458523585647, 5.269697119701937]}, {'id': 'Borborema_Provincia', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -73.98327050645874, 0, -0.00026949458523585647, 5.269697119701937]}, {'id': 'Cobertura_Cenozoica_Provincia', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [167483, 14

In [24]:
# Distance to specific landforms (MapBiomas, 2024)

# Rock outcrops
rock_path = 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/DISTANCE_C10_v1/distance_afloramento_rochoso_c10_v1'
rock_image = ee.Image(rock_path).rename('Distance_to_rock')

# Sand deposits (beaches, dunes, sandy areas)
sand_path = 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/DISTANCE_C10_v1/distance_praia_duna_areal_c10_v1'
sand_image = ee.Image(sand_path)

# Stack all distance to landform images into a single image
dist_image = rock_image.addBands(sand_image)
print(dist_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'Distance_to_rock', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -73.98327050645874, 0, -0.0002694945852358564, 5.269697119701938]}, {'id': 'praia_duna_areal_distance_m', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [167483, 144793], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694945852358564, 0, -73.98327050645874, 0, -0.0002694945852358564, 5.269697119701938]}], 'version': 1760670359225401, 'id': 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/DISTANCE_C10_v1/distance_afloramento_rochoso_c10_v1', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-44.36296908303576, -33.75136755494275], [-41.18937977207686, -33.75136754493187], [-35.19482217682088, -33.75136762345294], [-28.847318949928372, -33.75136735808534], [-28.847372557145455, 5.2698

In [ ]:
# Extract covariate values to sampling points
# This takes about 15 minutes to run...

# Stack the images into a single multiband image
static_image = (wrb_image # Soil Reference Groups
                .addBands([
                    black_image, # Black Soils (FAO, 2022)
                    soil_image, # SoilGrids
                    dem_image, # Land Surface Variables
                    geo_image, # Structural Provinces
                    dist_image # Distance to specific landforms
                ]))
                   
# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, static_image, scale=250)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
static_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Replace dash with underscores
static_df.columns = static_df.columns.str.replace('-', '_')
# Remove _mean from column names
static_df.columns = static_df.columns.str.replace('_mean', '')

# Print column names
print(static_df.columns)

# Check if number of rows of static_df is the same as that of soildata_xy
# If the number of rows is the same, the merge was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == static_df.shape[0], '\nThere are', static_df.shape[0], 'events')

Index(['Acrisols', 'Alisols', 'Amazonas_Solimoes_Provincia',
       'Amazonia_Provincia', 'Arenosols', 'Borborema_Provincia', 'Chernozems',
       'Cobertura_Cenozoica_Provincia',
       'Costeira_Margem_Continental_Provincia', 'Distance_to_rock',
       'Ferralsols', 'Gleysols', 'Gurupi_Provincia', 'Histosols', 'Leptosols',
       'Lixisols', 'Luvisols', 'Mantiqueira_Provincia',
       'Massa_d_agua_Provincia', 'Metamorficas_Subprovincia', 'Nitisols',
       'Parana_Provincia', 'Parecis_Provincia', 'Parnaiba_Provincia',
       'Phaeozems', 'Planosols', 'Plinthosols', 'Plutonicas_Subprovincia',
       'Podzols', 'Reconcavo_Tucano_Jatoba_Provincia', 'Regosols',
       'Sao_Francisco_Provincia', 'Sao_Luis_Provincia',
       'Sedimentares_Subprovincia', 'Sedimentos_Subprovincia', 'Stagnosols',
       'Tocantis_Provincia', 'Umbrisols', 'Vertisols',
       'Vulcanicas_Subprovincia', 'b1', 'bdod_0_5cm', 'bdod_100_200cm',
       'bdod_15_30cm', 'bdod_30_60cm', 'bdod_5_15cm', 'bdod_60_100cm',


In [26]:
# Compute summary statistics of the SoilGrids data
summary = static_df.describe()
print(summary)

           Acrisols       Alisols  Amazonas_Solimoes_Provincia  \
count  16569.000000  16569.000000                 16565.000000   
mean      18.563945      2.721589                     0.044914   
std       14.041612      5.186914                     0.207121   
min        0.000000      0.000000                     0.000000   
25%        8.000000      1.000000                     0.000000   
50%       15.000000      1.000000                     0.000000   
75%       27.000000      3.000000                     0.000000   
max       84.000000     70.000000                     1.000000   

       Amazonia_Provincia     Arenosols  Borborema_Provincia    Chernozems  \
count        16565.000000  16569.000000         16565.000000  16569.000000   
mean             0.147359      5.228982             0.041835      1.116302   
std              0.354474      6.324943             0.200218      1.771823   
min              0.000000      0.000000             0.000000      0.000000   
25%            

In [7]:
from datetime import datetime

# MapBiomas LULC Collection
# (this takes about 10 minutes to run)

# Import the MapBiomas Collection 9.0
collection = 'projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1'
mapbiomas_image = ee.Image(collection)

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, mapbiomas_image, scale=30)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

In [8]:
# Concatenate all DataFrames into a single DataFrame
mapbiomas_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Remove classification_ prefix from column names
mapbiomas_df.columns = mapbiomas_df.columns.str.replace('classification_', '')

# Get LULC class at the year of sampling (data_ano)
# Each column in the MapBiomas dataset represents a year. The column name is the year of the
# classification. The value is the class code. We need to extract the class code for the year of
# sampling, which is stored in the data_ano column.
# Step 1: Create a new column YEAR based on data_ano
mapbiomas_df['YEAR'] = mapbiomas_df['data_ano']
# Step 2: If YEAR is less than 1985, set it to 0 (no data)
mapbiomas_df.loc[mapbiomas_df['YEAR'] < 1985, 'YEAR'] = 0
# Step 3: Find the column index for each YEAR
lulc_idx = mapbiomas_df.columns.get_indexer(mapbiomas_df['YEAR'].astype(str))
# Step 4: Extract the class code for each row based on the data_ano column
lulc = mapbiomas_df.to_numpy()
lulc = lulc[range(len(lulc)), lulc_idx]
# Step 5: Convert the extracted class codes to strings and assign them to a new column lulc
mapbiomas_df['lulc'] = lulc.astype(str)
# Step 6: Drop the YEAR column
mapbiomas_df = mapbiomas_df.drop(columns = ['YEAR'])

# Some columns of mapbiomas_df are named with the year of the classification, ranging from 1985 to
# the present year. This columns need to be dropped.
# Drop columns with years as column names
current_year = datetime.now().year
years_to_drop = [str(year) for year in range(1985, current_year + 1)]
mapbiomas_df.drop(columns=years_to_drop, inplace=True, errors='ignore')

# Reclassify the land cover classes
forest_codes = ['1.0', '3.0', '4.0', '5.0', '6.0', '49.0']
nonforest_codes = ['10.0', '11.0', '12.0', '32.0', '29.0', '50.0']
pasture_codes = ['15.0']
agriculture_codes = [
    '14.0', '18.0', '19.0', '39.0', '20.0', '40.0', '62.0', '41.0', '36.0', '46.0', '47.0', '48.0',
    '21.0', '35.0']
forestry_codes = ['9.0']
nonvegetation_codes = ['22.0', '23.0', '24.0', '30.0', '25.0', '26.0', '33.0', '31.0', '27.0']
na_codes = ['0', '0.0', 'nan']
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forest_codes, 'forest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonforest_codes, 'nonforest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(pasture_codes, 'pasture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(agriculture_codes, 'agriculture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forestry_codes, 'forestry')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonvegetation_codes, 'nonvegetation')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(na_codes, pd.NA)

# Print summary of the land cover classes, including NA
print('\nDistribution of land cover/land use classes (including NA):')
print(mapbiomas_df['lulc'].value_counts(dropna=False))

# Check if number of rows of mapbiomas_df is the same as that of soildata_xy
# If the number of rows is the same, the sampling was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == mapbiomas_df.shape[0], '\nThere are', mapbiomas_df.shape[0], 'events')


Distribution of land cover/land use classes (including NA):
lulc
<NA>             4236
forest           3353
pasture          2668
agriculture      1691
nonforest        1023
nonvegetation     210
forestry          189
2024               11
Name: count, dtype: int64

There should be 13381 events: True 
There are 13381 events


In [9]:
# Geomorphometry 90 m

# Sample the terrain attributes
# projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m
# (this takes about 10 minutes to run)
dem = ee.Image("projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m")

# Initialize an empty list to store DataFrames
dem_dfs = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, dem, scale=90)
    sampled_df = geemap.ee_to_df(sampled_points)
    dem_dfs.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
dem_dataframe = pd.concat(dem_dfs, ignore_index=True)

# print column names
print(dem_dataframe.columns)

Index(['aspect', 'aspect_cosine', 'aspect_sine', 'convergence', 'coord_x',
       'coord_y', 'cti', 'data_ano', 'dataset_id', 'dev_magnitude',
       'dev_scale', 'dx', 'dxx', 'dxy', 'dy', 'dyy', 'eastness', 'elev_stdev',
       'geom', 'northness', 'observacao_id', 'pcurv', 'rough_magnitude',
       'roughness', 'slope', 'spi'],
      dtype='object')


In [10]:
# Compute summary statistics of the Geomorphometry data
summary = dem_dataframe.describe()
print(summary)

             aspect  aspect_cosine   aspect_sine   convergence       coord_x  \
count  13377.000000   13347.000000  13377.000000  13377.000000  13381.000000   
mean     182.927689      -0.020293     -0.019602     -2.944051    -52.668279   
std      102.681323       0.656209      0.653158     25.632127      8.480355   
min        0.074554      -0.999748     -0.999727    -85.449432    -73.783330   
25%       95.605042      -0.650972     -0.635149    -19.795198    -60.957222   
50%      185.982849      -0.047982     -0.049310     -3.478558    -53.581038   
75%      269.085083       0.607631      0.607330     14.010069    -45.800000   
max      359.995667       0.999829      0.999761     83.113701    -34.883300   

            coord_y           cti      data_ano  dev_magnitude     dev_scale  \
count  13381.000000  13377.000000  13381.000000   13377.000000  13377.000000   
mean     -16.487456     -0.270026   1993.556236       0.321840    803.499664   
std        9.062978      2.175182     1

In [11]:
# Merge data sampled from SoilGrids, MapBiomas, and Geomorphometry into soildata_df
# Keep all rows from soildata_df
merge_columns = ['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']
output_df = soildata_df.merge(soilgrids_df, on = merge_columns, how = 'left')
output_df = output_df.merge(mapbiomas_df, on = merge_columns, how = 'left')
output_df = output_df.merge(dem_dataframe, on = merge_columns, how = 'left')

# # Fill empty cells in the 'lulc' column with 'unknown'
# output_df['lulc'] = output_df['lulc'].fillna('unknown')

# Check if number of rows of output_df is the same as that of soildata_df
# If the number of rows is the same, the merge was successful
target = soildata_df.shape[0]
print(
  '\nThere should be', target, 'layers:',
  target == output_df.shape[0], '\nThere are', output_df.shape[0], 'layers')


There should be 29881 layers: True 
There are 29881 layers


In [12]:
# Write the output_df to a TXT file
# Field separator: tab. Decimal separator: comma
file_path = os.path.join(work_dir, 'data', '21_soildata_soc.txt')
output_df.to_csv(file_path, sep='\t', index=False)